**--- Inicio Colab ---**


In [ ]:
# Monto drive en colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**--- Fin Colab ---**


In [ ]:
import sys
sys.path.insert(1, '../FeatureEngineering')

import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation,Bidirectional,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
import util
import hyperparameter_tuning
embeddings = None
from hyperopt import hp,STATUS_OK

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [ ]:
def prepare_for_gru(df):
    processed = feature_builder.process_dataset(df,encoding_type='mean')
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

**Proceso el set de entrenamiento entero**

In [ ]:
processed, X_processed = prepare_for_gru(train_df)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Embeddings loaded!
Percentage of words covered in the embeddings = 0.4937444933920705
Embeddings loaded!
Percentage of words covered in the embeddings = 0.5879980378263476


**Hago split con train test split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

**Armo el modelo de GRU**


In [ ]:
def GRU():
    model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(32 , activation='relu'),
    tf.keras.layers.Dropout(0.4961765821337532),
    tf.keras.layers.Dense(1)])
    return model

In [ ]:
model = GRU()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 1024)              3523584   
_________________________________________________________________
dense_2 (Dense)              (None, 32)                32800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 3,556,417
Trainable params: 3,556,417
Non-trainable params: 0
_________________________________________________________________


**Entreno el modelo luego ejecuto prediccion**

In [ ]:
history = model.fit(X_train, y_train, epochs=30, shuffle=True, validation_split=0.2)

Epoch 1/30
128/128 [==============================] - 6s 45ms/step - loss: 0.6477 - accuracy: 0.5900 - val_loss: 0.6707 - val_accuracy: 0.5265
Epoch 2/30
128/128 [==============================] - 5s 40ms/step - loss: 0.6129 - accuracy: 0.6110 - val_loss: 0.6348 - val_accuracy: 0.5324
Epoch 3/30
128/128 [==============================] - 5s 40ms/step - loss: 0.5847 - accuracy: 0.6483 - val_loss: 0.6000 - val_accuracy: 0.6010
Epoch 4/30
128/128 [==============================] - 5s 40ms/step - loss: 0.5588 - accuracy: 0.6779 - val_loss: 0.5733 - val_accuracy: 0.6745
Epoch 5/30
128/128 [==============================] - 5s 41ms/step - loss: 0.5331 - accuracy: 0.7103 - val_loss: 0.5650 - val_accuracy: 0.6569
Epoch 6/30
128/128 [==============================] - 5s 41ms/step - loss: 0.5133 - accuracy: 0.7306 - val_loss: 0.5286 - val_accuracy: 0.7559
Epoch 7/30
128/128 [==============================] - 5s 41ms/step - loss: 0.4961 - accuracy: 0.7458 - val_loss: 0.5138 - val_accuracy: 0.7569

In [ ]:
prediction = model.predict_classes(X_test)
f1 = f1_score(y_test, prediction,average=None)
mean_f1 = (f1[0] + f1[1]) / 2
mean_f1

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


0.7871822871196327

**Vamos a probar con diferentes epochs y ver como varia el f1**

In [ ]:
model_50epochs = GRU()
model_50epochs.summary()
model_50epochs.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 1024)              3523584   
_________________________________________________________________
dense_4 (Dense)              (None, 32)                32800     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 3,556,417
Trainable params: 3,556,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model_50epochs.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2)

Epoch 1/50
128/128 [==============================] - 6s 44ms/step - loss: 0.6588 - accuracy: 0.5806 - val_loss: 0.6459 - val_accuracy: 0.5265
Epoch 2/50
128/128 [==============================] - 5s 39ms/step - loss: 0.6188 - accuracy: 0.5961 - val_loss: 0.6266 - val_accuracy: 0.5373
Epoch 3/50
128/128 [==============================] - 5s 39ms/step - loss: 0.5950 - accuracy: 0.6228 - val_loss: 0.6171 - val_accuracy: 0.5461
Epoch 4/50
128/128 [==============================] - 5s 40ms/step - loss: 0.5709 - accuracy: 0.6559 - val_loss: 0.5896 - val_accuracy: 0.6225
Epoch 5/50
128/128 [==============================] - 5s 39ms/step - loss: 0.5507 - accuracy: 0.6819 - val_loss: 0.5771 - val_accuracy: 0.6216
Epoch 6/50
128/128 [==============================] - 5s 39ms/step - loss: 0.5319 - accuracy: 0.7034 - val_loss: 0.5509 - val_accuracy: 0.6961
Epoch 7/50
128/128 [==============================] - 5s 40ms/step - loss: 0.5161 - accuracy: 0.7221 - val_loss: 0.5483 - val_accuracy: 0.6824

In [ ]:
prediction = model_50epochs.predict_classes(X_test)
f1 = f1_score(y_test, prediction,average=None)
mean_f1 = (f1[0] + f1[1]) / 2
mean_f1

0.8004597242468909

In [ ]:
model_100epochs = GRU()
model_100epochs.summary()
model_100epochs.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 1024)              3523584   
_________________________________________________________________
dense_6 (Dense)              (None, 32)                32800     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 3,556,417
Trainable params: 3,556,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model_100epochs.fit(X_train, y_train, epochs=100, shuffle=True, validation_split=0.2)

Epoch 1/100
128/128 [==============================] - 6s 43ms/step - loss: 0.6751 - accuracy: 0.5779 - val_loss: 0.6445 - val_accuracy: 0.5265
Epoch 2/100
128/128 [==============================] - 5s 38ms/step - loss: 0.6153 - accuracy: 0.6123 - val_loss: 0.6199 - val_accuracy: 0.5529
Epoch 3/100
128/128 [==============================] - 5s 38ms/step - loss: 0.5907 - accuracy: 0.6493 - val_loss: 0.5976 - val_accuracy: 0.6167
Epoch 4/100
128/128 [==============================] - 5s 37ms/step - loss: 0.5648 - accuracy: 0.6860 - val_loss: 0.5768 - val_accuracy: 0.6265
Epoch 5/100
128/128 [==============================] - 5s 38ms/step - loss: 0.5425 - accuracy: 0.7044 - val_loss: 0.5545 - val_accuracy: 0.6853
Epoch 6/100
128/128 [==============================] - 5s 38ms/step - loss: 0.5221 - accuracy: 0.7235 - val_loss: 0.5330 - val_accuracy: 0.7539
Epoch 7/100
128/128 [==============================] - 5s 37ms/step - loss: 0.5034 - accuracy: 0.7414 - val_loss: 0.5318 - val_accuracy:

In [ ]:
prediction = model_100epochs.predict_classes(X_test)
f1 = f1_score(y_test, prediction,average=None)
mean_f1 = (f1[0] + f1[1]) / 2
mean_f1

0.8089920687376074

**Búsqueda de parámetros con Hyperopt (bayesian)**

In [ ]:
# Parametros que voy a ir probando
space = {'units': hp.choice('units',[16,32,64,128,256,512]),
        'units2': hp.choice('units2',[16,32,64,128,256,512]),
        'dropout': hp.uniform('dropout',0,1),
        'activation': hp.choice('activation',['relu','sigmoid'])
        }

In [ ]:
def gru(params):
  with tf.device('/GPU:0'):
    print("Probando parametros: ")
    print("units: %s, units2: %s, dropout: %s, activation: %s" % (params['units'],params['units2'],params['dropout'],params['activation']))
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(params['units']),input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(tf.keras.layers.Dense(params['units2'], activation=params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout']))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
    
    result = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2,verbose=0)
    prediction = model.predict_classes(X_test)
    f1 = f1_score(y_test, prediction,average=None)
    mean_f1 = (f1[0] + f1[1]) / 2
    mean_f1
    print('\nScore:',mean_f1)
    return {'loss': -mean_f1, 'status': STATUS_OK, 'model': model}

In [ ]:
import importlib
importlib.reload(hyperparameter_tuning)
hyperparameter_tuning.bayesian_optimization(gru,space,max_evals=300)

Probando parametros: 
units: 32, units2: 64, dropout: 0.802607417233141, activation: relu
  0%|          | 0/300 [00:06<?, ?it/s, best loss: ?]WARNING:tensorflow:From <ipython-input-11-ae8cf7ba9c66>:17: Sequential.predict_classes (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

Score:
0.7833692061698889
Probando parametros: 
units: 16, units2: 64, dropout: 0.902087952722073, activation: sigmoid

Score:
0.5379119219302048
Probando parametros: 
units: 16, units2: 64, dropout: 0.36590291353893856, activation: relu

Score:
0.8009390646679453
Probando parametros: 
units: 256, units2: 512, dropout: 0.18748677097

Si bien fallo de recorrer las 300 vueltas de búsqueda bayesiana (esta operación requiere bastante RAM y ni siquiera colab con GPU pudo llegar a tal iteración, al menos probando de esta forma) se obtuvo que hasta la número 224 los mejores hiperparámetros fueron los siguientes:

units: 512, units2: 32, dropout: 0.4961765821337532, activation: relu

Esto en terminos de nuestro mapa de parámetros sería:

space = {'units': 512,
        'units2': 32,
        'dropout': 0.4961765821337532,
        'activation':relu}

Voy a usar estos hiperparámetros para correr el export de kaggle

**Kaggle export**

In [ ]:
processed, X_processed = prepare_for_gru(train_df)

Embeddings loaded!
Percentage of words covered in the embeddings = 0.4937444933920705
Embeddings loaded!
Percentage of words covered in the embeddings = 0.5879980378263476


In [ ]:
y_train = train_df['target'].values

In [ ]:
def GRU():
  with tf.device('/GPU:0'):
    model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512),input_shape=(X_processed.shape[1], X_processed.shape[2])),
    tf.keras.layers.Dense(32 , activation='relu'),
    tf.keras.layers.Dropout(0.4961765821337532),
    tf.keras.layers.Dense(1)])
    return model

In [ ]:
model = GRU()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_10 (Bidirectio (None, 1024)              3523584   
_________________________________________________________________
dense_20 (Dense)             (None, 32)                32800     
_________________________________________________________________
dropout_10 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 33        
Total params: 3,556,417
Trainable params: 3,556,417
Non-trainable params: 0
_________________________________________________________________


Se probaron entre 30, 50 y 100 epochs con submissions de 0.80355, 0.80907 y 0.79773 respectivamente, por lo que se realizaran los entrenamientos con 50 epochs.

In [ ]:
history = model.fit(X_processed, y_train, epochs=50, shuffle=False, validation_split=0.0)

Epoch 1/50
238/238 [==============================] - 9s 38ms/step - loss: 0.6781 - accuracy: 0.5756
Epoch 2/50
238/238 [==============================] - 9s 38ms/step - loss: 0.6338 - accuracy: 0.5911
Epoch 3/50
238/238 [==============================] - 9s 37ms/step - loss: 0.6111 - accuracy: 0.6254
Epoch 4/50
238/238 [==============================] - 9s 38ms/step - loss: 0.5891 - accuracy: 0.6503
Epoch 5/50
238/238 [==============================] - 9s 38ms/step - loss: 0.5628 - accuracy: 0.6844
Epoch 6/50
238/238 [==============================] - 9s 38ms/step - loss: 0.5463 - accuracy: 0.6954
Epoch 7/50
238/238 [==============================] - 9s 37ms/step - loss: 0.5238 - accuracy: 0.7066
Epoch 8/50
238/238 [==============================] - 9s 37ms/step - loss: 0.5068 - accuracy: 0.7289
Epoch 9/50
238/238 [==============================] - 9s 37ms/step - loss: 0.4916 - accuracy: 0.7461
Epoch 10/50
238/238 [==============================] - 9s 37ms/step - loss: 0.4797 - accura

In [ ]:
ids = test_df['id']

processed, final_test = prepare_for_gru(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

Embeddings loaded!
Percentage of words covered in the embeddings = 0.5707598689343111
Embeddings loaded!
Percentage of words covered in the embeddings = 0.6603057058767683


,target
id,
0,1
2,1
3,1
9,1
11,1
...,...
10861,1
10865,1
10868,1


In [ ]:
final_df['target'].value_counts()

0    1947
1    1316
Name: target, dtype: int64

In [ ]:
final_df.to_csv('gru_new_2.csv')